# Generate images from logos with Stable Diffusion fine-tuning

---
Imagine having an application that easily generates an image on how your product logo can be put on the wall or on banners, and see how they look like, probably for an ideation and visualization of a PoC. While Stable Diffusion model can generate images, it may not know how to draw your logo / icon yet. So in this lab, we will see how we can fine-tune a Stable Diffusion model for this purpose. We will use some image samples and their captions. Please look at the training_images folder.

In this demo notebook, we demonstrate how to use the JumpStart APIs to fine-tune the Stable Diffusion model. The training script is based on Dreambooth, with some modification to allow multiple objects training and to log the loss into CloudWatch metrics so that you can plot the loss vs time.

Note: To run this notebook, you would need the **`Data Science 3.0`** kernel in SageMaker Studio. For performing local inference (step 4), a [GPU-attached instance type](https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-available-instance-types.html#notebooks-resources-gpu) such as `ml.g4dn.xlarge`, `ml.g5.xlarge`, `ml.g4dn.2xlarge` or `ml.g5.2xlarge` is required. Otherwise, the notebook should work with most SageMaker Studio instance types.

---

0. [Set Up](#set-up)

1. [Deploy and test the SageMaker JumpStart model](#section-1)

2. [Fine-tune the pre-trained model on a custom dataset](#section-2)

3. [Further tuning the model with hyperparameter optimization](#section-3)

4. [Test locally (only if using GPU-attached instance for kernel)](#section-4)

## Set Up

In [ ]:
%load_ext autoreload
%autoreload 2

import botocore
import sagemaker, boto3, json
import matplotlib.pyplot as plt
from sagemaker.experiments.run import Run
from sagemaker.utils import unique_name_from_base
from sagemaker import get_execution_role
import os
import time

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
training_bucket = sess.default_bucket()

# If uploading to a different folder, change this variable.
local_training_dataset_folder = "training_images"
if not os.path.exists(local_training_dataset_folder):
    os.mkdir(local_training_dataset_folder)

In [ ]:
#import custom functions from utils.py
from utils import query, parse_response, query_endpoint_with_json_payload, parse_response_multiple_images, display_img_and_prompt, image_grid

In [ ]:
# Instance prompt is fed into the training script via dataset_info.json present in the training folder. Here, we write that file.
import os
import json
instance_prompt = "AWS Lambda"
with open(os.path.join(local_training_dataset_folder, "dataset_info.json"), "w") as f:
    f.write(json.dumps({"instance_prompt": instance_prompt}))

Let's first take a look at the data. The caption is also important. For now we are using images of the `AWS Lambda` icon for the training data.

In [ ]:
from PIL import Image

image_paths = os.listdir(local_training_dataset_folder)
num_images = len(image_paths)-1
rows, cols = (int(num_images/4)+1,4)
images = []
for path in image_paths:
    if path == ".ipynb_checkpoints" or path == "dataset_info.json": 
        continue
    image_path = f"{local_training_dataset_folder}/{path}"
    images.append(Image.open(image_path))
image_grid(images, rows, cols)

In [ ]:
experiment_name = unique_name_from_base("logos-to-images", max_length=32)
run_name = str(time.ctime()).replace(" ","-").replace(":","-")
project_s3_path = f"s3://{training_bucket}/{experiment_name}/{run_name}"

In [ ]:
dataset_s3_path = f"{project_s3_path}/dataset/"

!aws s3 cp --recursive $local_training_dataset_folder $dataset_s3_path

## <a id="section-1">1. Deploy and test the SageMaker JumpStart model<a>

In [ ]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

# Retrieves all Text-to-Image generation models.
filter_value = "task == txt2img"
txt2img_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=txt2img_models,
    value="model-txt2img-stabilityai-stable-diffusion-v2-1-base",
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(model_dropdown)

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

inference_instance_type = "ml.g5.2xlarge"
model_id = model_dropdown.value
endpoint_name = unique_name_from_base(f"jumpstart-{model_id}")

jumpstart_model = JumpStartModel(model_id=model_id)
predictor = jumpstart_model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

In [ ]:
# Simple text input
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
query_response = query(predictor, prompt)
img, prmpt = query_response["generated_image"], query_response["prompt"] #note that output is different from inference using trained models
display_img_and_prompt(img, prmpt)

In [ ]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

## <a id="section-2">2. Fine-tune the pre-trained model on a custom dataset<a>

In [ ]:
# model_version="*" fetches the latest version of the model
train_model_id, train_model_version = model_dropdown.value, "*"
train_scope = "training"

In [ ]:
from sagemaker import image_uris, model_uris, script_uris

# Tested with ml.g4dn.2xlarge (16GB GPU memory) and ml.g5.2xlarge (24GB GPU memory) instances. Other instances may work as well.
# If ml.g5.2xlarge instance type is available, please change the following instance type to speed up training.
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script. This contains all the necessary files including data processing, model training etc.
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)

# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

In [ ]:
output_bucket = training_bucket

s3_output_location = f"{project_s3_path}/output/"

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["max_steps"] = "300"
hyperparameters["epochs"] = "10"
print(hyperparameters)

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{train_model_id}-transfer-learning")

# Create SageMaker Estimator instance
sd_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

In [ ]:
%%time

# Launch a SageMaker Training job by passing s3 path of the training data
sd_estimator.fit({"training": dataset_s3_path}, logs=True)

In [ ]:
inference_instance_type = "ml.g5.2xlarge"
inference_scope = "inference"

# Retrieve the docker image for inference
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None, 
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=inference_scope,
    instance_type=inference_instance_type,
)

endpoint_name = name_from_base(f"jumpstart-example-{train_model_id}-transfer-learning")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
predictor = sd_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    image_uri=deploy_image_uri,
    endpoint_name=endpoint_name,
)

In [ ]:
# Simple text input
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
query_response = query(predictor, prompt)
img, prmpt = parse_response(query_response)
display_img_and_prompt(img, prmpt)

In [ ]:
# Payload input
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
negative_prompt = None
payload = {"prompt": prompt, 
           "negative_prompt": negative_prompt, 
           "seed": 16,
           "num_images_per_prompt": 1,
           "num_inference_steps": 50,
           "guidance_scale": 10,
          }

query_response = query_endpoint_with_json_payload(
    predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
# Payload input with negative prompt
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
negative_prompt = "bubble"
payload = {"prompt": prompt, 
           "negative_prompt": negative_prompt, 
           "seed": 16,
           "num_images_per_prompt": 1,
           "num_inference_steps": 50,
           "guidance_scale": 10,
          }

query_response = query_endpoint_with_json_payload(
    predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
# Delete the SageMaker endpoint
#predictor.delete_model()
#predictor.delete_endpoint()

## <a id="section-3">3. Further tuning the model with hyperparameter optimization<a>

In [ ]:
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import HyperparameterTuner

tuning_job_name = experiment_name

hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(1e-6, 3e-6, "Linear"),
    "max_steps": IntegerParameter(50, 400, "Linear"),
    "epochs": IntegerParameter(10, 30, "Linear"),
}

In [ ]:
sd_estimator.set_hyperparameters(compute_fid="False")

tuner_parameters = {
    "estimator": sd_estimator,
    "metric_definitions": [{"Name": "fid_score", "Regex": "fid_score=([-+]?\\d\\.?\\d*)"},
                          {"Name": "train_avg_loss", "Regex": "train_avg_loss=([-+]?\\d\\.?\\d*)"}],
    "objective_metric_name": "train_avg_loss",
    "objective_type": "Minimize",
    "hyperparameter_ranges": hyperparameter_ranges,
    "max_jobs": 10,
    "max_parallel_jobs": 1,
    "strategy": "Bayesian",
    "base_tuning_job_name": {training_job_name},
}

tuner = HyperparameterTuner(**tuner_parameters)

In [ ]:
%%time

tuner.fit(
    {"training": dataset_s3_path}, 
    job_name=tuning_job_name,
    logs=True)

**Note**

If the tuner times out, rerun the first cell after [Deploy and test the SageMaker JumpStart model](#Section-1), uncomment the variable `tuning_job_name` and copy the tuning job name before running the following cell.

In [ ]:
# tuning_job_name = #specify tuning job name

sm = boto3.client('sagemaker')
hpo_jobs = sm.list_training_jobs_for_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name,
    MaxResults=100,
    SortBy='FinalObjectiveMetricValue',
    SortOrder='Ascending')

joblist = []
hposummaries = hpo_jobs['TrainingJobSummaries']

for job in hposummaries[:]:
    TrainingJobName=job['TrainingJobName']
    job_descr = sm.describe_training_job(TrainingJobName=TrainingJobName)
    metrics = {m['MetricName']:  m['Value'] for m in job_descr['FinalMetricDataList']}
    hyperparams = job['TunedHyperParameters']
    joblist.append({"Training job name": TrainingJobName, "Metrics": metrics, "Hyperparameters": hyperparams})
print(*joblist,sep='\n')

**Note**

Do not run the following cell if the tuner has timed out.

In [ ]:
##Run if tuner has not timed out, else use the following cell

inference_instance_type = "ml.g5.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)

endpoint_name = f"jumpstart-FT-{tuning_job_name}"

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = tuner.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    image_uri=deploy_image_uri,
    endpoint_name=endpoint_name,
)

**Note** 

Uncomment and use the following cell if the tuner has timed out

In [ ]:
# ##Uncomment and run if tuner has timed out
# from sagemaker.estimator import Estimator
# from sagemaker import image_uris, model_uris, script_uris


# train_model_id, train_model_version = model_dropdown.value, "*"

# inference_instance_type = "ml.g5.xlarge"

# # Retrieve the inference docker container uri
# deploy_image_uri = image_uris.retrieve(
#     region=None,
#     framework=None,  # automatically inferred from model_id
#     image_scope="inference",
#     model_id=train_model_id,
#     model_version=train_model_version,
#     instance_type=inference_instance_type,
# )

# job_name = joblist[0]['Training job name']

# endpoint_name = f"jumpstart-FT-{job_name}"

# finetuned_estimator = Estimator.attach(job_name)
# finetuned_predictor = finetuned_estimator.deploy(
#     instance_type = inference_instance_type,
#     initial_instance_count = 1,
#     image_uri=deploy_image_uri,
#     endpoint_name=endpoint_name
# )

In [ ]:
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
negative_prompt = None
payload = {"prompt": prompt, 
           "negative_prompt": negative_prompt, 
           "seed": 16,
           "num_images_per_prompt": 1,
           "num_inference_steps": 50,
           "guidance_scale": 10,
          }

query_response = query_endpoint_with_json_payload(
    finetuned_predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
negative_prompt = "bubble, black border"
payload = {"prompt": prompt, 
           "negative_prompt": negative_prompt, 
           "seed": 16,
           "num_images_per_prompt": 1,
           "num_inference_steps": 50,
           "guidance_scale": 10,
          }


query_response = query_endpoint_with_json_payload(
    finetuned_predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
# Payload input with negative prompt
prompt = "A photo of Singapore urban park, balloon with AWS Lambda logo, realistic, cinematic, bright, 8k"
payload = {"prompt": prompt, 
           "negative_prompt": "dark, cropped, clipped, deformed, duplicates", 
           "seed": 7,
           "num_images_per_prompt": 1,
           "num_inference_steps": 75,
           "guidance_scale": 15,
          }

query_response = query_endpoint_with_json_payload(
    finetuned_predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
# Delete the SageMaker endpoint
#finetuned_predictor.delete_model()
#finetuned_predictor.delete_endpoint()

### Select a specific training job's model to deploy for inference

In [ ]:
# inference_instance_type = "ml.g4dn.xlarge"

job_name = joblist[3]['Training job name']
# job_name = #specify training job name

endpoint_name = f"jumpstart-FT-{job_name}"

attached_estimator = Estimator.attach(job_name)
attached_predictor = attached_estimator.deploy(
    instance_type = inference_instance_type,
    initial_instance_count = 1,
    image_uri=deploy_image_uri,
    endpoint_name=endpoint_name
)

In [ ]:
prompt = "AWS Lambda on a rock, in a dream landscape, cinematic composition, futuristic, in the mountains, high quality"
negative_prompt = "bubble, black border"
payload = {"prompt": prompt, 
           "negative_prompt": negative_prompt, 
           "seed": 16,
           "num_images_per_prompt": 1,
           "num_inference_steps": 50,
           "guidance_scale": 10,
          }


query_response = query_endpoint_with_json_payload(
    attached_predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
# Payload input with negative prompt
prompt = "A photo of Singapore urban park, balloon with AWS Lambda logo, realistic, cinematic, bright, 8k"
payload = {"prompt": prompt, 
           "negative_prompt": "dark, cropped, clipped, deformed, duplicates", 
           "seed": 7,
           "num_images_per_prompt": 1,
           "num_inference_steps": 75,
           "guidance_scale": 15,
          }

query_response = query_endpoint_with_json_payload(
    attached_predictor, payload, "application/json", "application/json"
)
generated_images, prompt = parse_response_multiple_images(query_response)

for img in generated_images:
    display_img_and_prompt(img, prompt)

In [ ]:
# Delete the SageMaker endpoint
#attached_predictor.delete_model()
#attached_predictor.delete_endpoint()

## <a id="section-4">4. Test locally (only if using GPU-attached instance for kernel)<a>

Uncomment and run the following cells to test the model locally.

In [ ]:
# import sagemaker

# training_job_name = joblist[0]['Training job name']
# # training_job_name = sd_estimator._current_job_name
# # training_job_name = #specify training job name
# training_job = sagemaker.estimator.Estimator.attach(training_job_name)
# model_uri = training_job.model_data
# !mkdir -p test_models
# !aws s3 cp $model_uri ./test_models/
# !cd test_models && tar -xzf model.tar.gz && cd ..

In [ ]:
# !pip install diffusers==0.10.2 transformers scipy ftfy accelerate
# from diffusers import StableDiffusionPipeline
# import torch
    
# # torch.cuda.empty_cache()
# # torch.cuda.ipc_collect()

# generator = torch.Generator("cuda").manual_seed(777)
# pipe = StableDiffusionPipeline.from_pretrained("./test_models/")
# pipe.to("cuda")

In [ ]:
# neg_prompt = "faces, eyes, animals, men, women"
# neg_prompts = [neg_prompt,neg_prompt]
# seed = 11
# generator = torch.Generator(device=0).manual_seed(seed)
# all_prompts = [
#     "AWS Lambda poster on brick wall, nail, sky, sunny day",
#     "A large painting of AWS Lambda on a wall above a bed, modern house, urban, bright lighting"
# ]
# images = pipe(all_prompts, negative_prompt=neg_prompts, guidance_scale= 50, height=512, width=512, num_inference_steps=75, generator=generator).images
# image_grid(images, rows=1, cols=2)
